# Conversational Interface - Chatbot with Claude LLM

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

In this notebook, we will build a chatbot using the Foundation Models (FMs) in Amazon Bedrock. For our use-case we use Claude V3 Sonnet as our foundation models.  For more details refer to [Documentation](https://aws.amazon.com/bedrock/claude/). The ideal balance between intelligence and speed—particularly for enterprise workloads. It excels at complex reasoning, nuanced content creation, scientific queries, math, and coding. Data teams can use Sonnet for RAG, as well as search and retrieval across vast amounts of information while sales teams can leverage Sonnet for product recommendations, forecasting, and targeted marketing. 

## Overview

Conversational interfaces such as chatbots and virtual assistants can be used to enhance the user experience for your customers.Chatbots uses natural language processing (NLP) and machine learning algorithms to understand and respond to user queries. Chatbots can be used in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. They can be accessed through various channels such as websites, social media platforms, and messaging apps.


## Chatbot using Amazon Bedrock

![Amazon Bedrock - Conversational Interface](./images/chatbot_bedrock.png)


## Use Cases

1. **Chatbot (Basic)** - Zero Shot chatbot with a FM model
2. **Chatbot using prompt** - template(Langchain) - Chatbot with some context provided in the prompt template
3. **Chatbot with persona** - Chatbot with defined roles. i.e. Career Coach and Human interactions
4. **Contextual-aware chatbot** - Passing in context through an external file by generating embeddings.

## Langchain framework for building Chatbot with Amazon Bedrock
In Conversational interfaces such as chatbots, it is highly important to remember previous interactions, both at a short term but also at a long term level.

LangChain provides memory components in two forms. First, LangChain provides helper utilities for managing and manipulating previous chat messages. These are designed to be modular and useful regardless of how they are used. Secondly, LangChain provides easy ways to incorporate these utilities into chains.
It allows us to easily define and interact with different types of abstractions, which make it easy to build powerful chatbots.

## Building Chatbot with Context - Key Elements

The first process in a building a contextual-aware chatbot is to **generate embeddings** for the context. Typically, you will have an ingestion process which will run through your embedding model and generate the embeddings which will be stored in a sort of a vector store. In this example we are using Titan Embeddings model for this

![Embeddings](./images/embeddings_lang.png)

Second process is the user request orchestration , interaction,  invoking and returing the results

![Chatbot](./images/chatbot_lang.png)

## Architecture [Context Aware Chatbot]
![4](./images/context-aware-chatbot.png)


## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](../00_Prerequisites/bedrock_basics.ipynb) notebook. ⚠️ ⚠️ ⚠️ Then run these installs below


In [2]:
%pip install -U --no-cache-dir boto3
%pip install -U --no-cache-dir  \
    "langchain>=0.1.11" \
    sqlalchemy -U \
    "faiss-cpu>=1.7,<2" \
    "pypdf>=3.8,<4" \
    pinecone-client==2.2.4 \
    apache-beam==2.52. \
    tiktoken==0.5.2 \
    "ipywidgets>=7,<8" \
    matplotlib==3.8.2 \
    anthropic==0.9.0
%pip install -U --no-cache-dir transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import warnings

from io import StringIO
import sys
import textwrap
import os
from typing import Optional

# External Dependencies:
import boto3
from botocore.config import Config

warnings.filterwarnings('ignore')

def print_ww(*args, width: int = 100, **kwargs):
    """Like print(), but wraps output to `width` characters (default 100)"""
    buffer = StringIO()
    try:
        _stdout = sys.stdout
        sys.stdout = buffer
        print(*args, **kwargs)
        output = buffer.getvalue()
    finally:
        sys.stdout = _stdout
    for line in output.splitlines():
        print("\n".join(textwrap.wrap(line, width=width)))
        



def get_bedrock_client(
    assumed_role: Optional[str] = None,
    region: Optional[str] = None,
    runtime: Optional[bool] = True,
):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides

    Parameters
    ----------
    assumed_role :
        Optional ARN of an AWS IAM role to assume for calling the Bedrock service. If not
        specified, the current active credentials will be used.
    region :
        Optional name of the AWS Region in which the service should be called (e.g. "us-east-1").
        If not specified, AWS_REGION or AWS_DEFAULT_REGION environment variable will be used.
    runtime :
        Optional choice of getting different client to perform operations with the Amazon Bedrock service.
    """
    if region is None:
        target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
    else:
        target_region = region

    print(f"Create new client\n  Using region: {target_region}")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        response = sts.assume_role(
            RoleArn=str(assumed_role),
            RoleSessionName="langchain-llm-1"
        )
        print(" ... successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    bedrock_client = session.client(
        service_name=service_name,
        config=retry_config,
        **client_kwargs
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client

In [4]:
import json
import os
import sys

import boto3




# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region='us-west-2' #os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


## Chatbot (Basic - without context)

We use [CoversationChain](https://python.langchain.com/en/latest/modules/models/llms/integrations/bedrock.html?highlight=ConversationChain#using-in-a-conversation-chain) from LangChain to start the conversation. We also use the [ConversationBufferMemory](https://python.langchain.com/en/latest/modules/memory/types/buffer.html) for storing the messages. We can also get the history as a list of messages (this is very useful in a chat model).

Chatbots needs to remember the previous interactions. Conversational memory allows us to do that. There are several ways that we can implement conversational memory. In the context of LangChain, they are all built on top of the ConversationChain.

**Note:** The model outputs are non-deterministic

In [10]:
#modelId = "anthropic.claude-3-sonnet-20240229-v1:0" #"anthropic.claude-v2"
modelId = "anthropic.claude-v2:1"

messages=[
    { 
        "role":'user', 
        "content":[{
            'type':'text',
            'text': "What is quantum mechanics? "
        }]
    },
    { 
        "role":'assistant', 
        "content":[{
            'type':'text',
            'text': "It is a branch of physics that describes how matter and energy interact with discrete energy values "
        }]
    },
    { 
        "role":'user', 
        "content":[{
            'type':'text',
            'text': "Can you explain a bit more about discrete energies?"
        }]
    }
]
body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 100,
            "messages": messages,
            "temperature": 0.5,
            "top_p": 0.9
        }  
    )  
    
response = boto3_bedrock.invoke_model(body=body, modelId=modelId)
response_body = json.loads(response.get('body').read())
print(response_body)


def test_sample_claude_invoke(prompt_str,boto3_bedrock ):
    #modelId = "anthropic.claude-3-sonnet-20240229-v1:0" #"anthropic.claude-v2"
    modelId = "anthropic.claude-v2:1"
    messages=[{ 
        "role":'user', 
        "content":[{
            'type':'text',
            'text': prompt_str
        }]
    }]
    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 100,
            "messages": messages,
            "temperature": 0.5,
            "top_p": 0.9
        }  
    )  
    response = boto3_bedrock.invoke_model(body=body, modelId=modelId)
    response_body = json.loads(response.get('body').read())
    return response_body


test_sample_claude_invoke("what is quantum mechanics", boto3_bedrock)   

{'id': 'msg_bdrk_01CGBnsANEMYnnh4n2YUDqAT', 'type': 'message', 'role': 'assistant', 'model': 'claude-2.1', 'stop_sequence': None, 'usage': {'input_tokens': 49, 'output_tokens': 100}, 'content': [{'type': 'text', 'text': 'Sure, the key idea in quantum mechanics is that certain physical quantities can only take on certain discrete values, rather than continuous values. Here are some key points about discrete energies:\n\n- Energy is quantized in quantum systems - it can only take on certain specific values and cannot have any in-between values. For example, an electron orbiting an atom has specific energy levels it can occupy.\n\n- Photons or packets of light also have discrete energies related to their wavelength or frequency by'}], 'stop_reason': 'max_tokens'}


{'id': 'msg_bdrk_01LU6W9pSGW4kpvVSxznmT5H',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-2.1',
 'stop_sequence': None,
 'usage': {'input_tokens': 13, 'output_tokens': 100},
 'content': [{'type': 'text',
   'text': 'Quantum mechanics is a fundamental theory in physics that describes the behavior of matter and energy at the subatomic level. Here are some key points about quantum mechanics:\n\n- It explains phenomena that classical physics cannot, such as the wave-particle duality of matter and quantum entanglement. At the quantum scale, particles can behave like waves and vice versa.\n\n- It introduces concepts like quantum superposition and quantum uncertainty. Particles do not have defined properties until they are measured, only probabilities that collapse'}],
 'stop_reason': 'max_tokens'}

In [14]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm_chat = BedrockChat(
    #model_id="anthropic.claude-3-sonnet-20240229-v1:0", 
    model_id = "anthropic.claude-v2:1",
    model_kwargs={"temperature": 0.1},
    region_name="us-west-2"
)

memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm_chat, verbose=True, memory=memory
)

conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


'Hello!'

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
#from langchain_openai import ChatOpenAI

modelId = "anthropic.claude-3-sonnet-20240229-v1:0" #"anthropic.claude-v2"
cl_llm = BedrockChat(
    model_id=modelId,
    client=boto3_bedrock,
    #model_kwargs={"max_tokens_to_sample": 100},
    model_kwargs={"temperature": 0.1},
)
memory = ConversationBufferMemory()
conversation = ConversationChain( llm=cl_llm, verbose=True, memory=memory)


prompt_t = ChatPromptTemplate.from_messages([("human", "Explain this  {question}.")])
chain_t = prompt_t | cl_llm | StrOutputParser()

In [17]:
#print(dir(cl_llm))

#modelId = "anthropic.claude-3-sonnet-20240229-v1:0" #"anthropic.claude-v2"
modelId = "anthropic.claude-v2:1"

messages=[
    { 
        "role":'user', 
        "content":[{
            'type':'text',
            'text': "What is quantum mechanics? "
        }]
    },
]
body_json=json.dumps(
        {
            #"anthropic_version": "bedrock-2023-05-31",
            #"max_tokens": 100,
            "messages": messages,
            "temperature": 0.5,
            "top_p": 0.9
        }  
    )  
cl_llm = BedrockChat(
    model_id=modelId,
    client=boto3_bedrock,
    #model_kwargs={"max_tokens_to_sample": 100},
    model_kwargs={"temperature": 0.1, 'max_tokens': 100},
    
)
cl_llm.predict(body_json)


'Quantum mechanics is a fundamental theory in physics that describes the behavior of matter and energy at the scale of atoms and subatomic particles. Here are some key points about quantum mechanics:\n\n- It explains phenomena that classical physics cannot, such as the discrete nature of energy levels in atoms, wave-particle duality, quantum entanglement, and quantum tunneling. \n\n- It is based on the idea that energy, momentum, angular momentum, etc. can only have certain discrete or quantized values.'

What happens here? We said "Hi there!" and the model spat out a several conversations. This is due to the fact that the default prompt used by Langchain ConversationChain is not well designed for Claude. An [effective Claude prompt](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design) should contain `\n\nHuman:` at the beginning and also contain `\n\nAssistant:` in the prompt sometime after the `\n\nHuman:` (optionally followed by other text that you want to [put in Claude's mouth](https://docs.anthropic.com/claude/docs/human-and-assistant-formatting#use-human-and-assistant-to-put-words-in-claudes-mouth)). Let's fix this.

To learn more about how to write prompts for Claude, check [Anthropic documentation](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design).

## Chatbot using prompt template (Langchain)

LangChain provides several classes and functions to make constructing and working with prompts easy. We are going to use the [PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html) class to construct the prompt from a f-string template. 

In [18]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

# turn verbose to true to see the full logs and documents
#modelId = "anthropic.claude-3-sonnet-20240229-v1:0" #"anthropic.claude-v2"
modelId = "anthropic.claude-v2:1"
cl_llm = BedrockChat(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs={"temperature": 0.1, 'max_tokens': 100},
)
conversation= ConversationChain(
    llm=cl_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude
claude_prompt = PromptTemplate.from_template("""

Human: The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
{history}
</conversation_history>

Here is the human's next reply:
<human_reply>
{input}
</human_reply>

Assistant:
""")

conversation.prompt = claude_prompt

#print_ww(conversation.predict(input="Hi there!"))

#### New Questions

Model has responded with intial message, let's ask few questions

In [19]:
print_ww(conversation.predict(input="Give me a few tips on how to start a new garden."))

Here are a few tips for starting a new garden:

Choose the right location. Pick a spot that gets at least 6 hours of sun per day and has good
drainage. Avoid planting under trees or eaves where plants won't get enough sun.

Prepare the soil properly. Remove any weeds or grass in the area, then mix in compost or other
organic material to enrich the soil before planting. You can also get a soil test done to see if you
need to adjust pH


#### Build on the questions

Let's ask a question without mentioning the word garden to see if model can understand previous conversation

In [20]:
print_ww(conversation.predict(input="Cool. Will that work with tomatoes?"))

Certainly, those tips would work well for starting a tomato garden. Here are some additional tomato-
specific tips:

- Choose tomato varieties suited to your growing season and climate. Determinate varieties are best
for container gardening or if you want all the fruit to ripen at once. Indeterminate do better
staked up and will produce fruit throughout the season.

- Amend the soil with compost or fertilizer formulated for vegetables. Tomatoes need nutrient-rich
soil and


#### Finishing this conversation

In [21]:
print_ww(conversation.predict(input="That's all, thank you!"))

Unfortunately I do not have enough context to continue the conversation in a meaningful way or
provide a substantive response. As the AI indicated previously, it would truthfully state if it does
not know the answer to a question. Since there is no clear question being asked and the human has
indicated the conversation is over, there does not seem to be a productive way for me to respond.
Please let me know if you have another question or topic you would like to discuss.


Claude is still really talkative. Try changing the prompt to make Claude provide shorter answers.

In [26]:
modelId = "anthropic.claude-v2:1"
cl_llm = BedrockChat(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs={"temperature": 0.1, 'max_tokens': 100},
)
conversation2= ConversationChain(
    llm=cl_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude
claude_prompt2 = PromptTemplate.from_template("""

Human: The following is an informative conversation between a human and an AI.
The AI is not talkative and answers questions with a maximum of 3 sentences from its context. The AI does not elaborate or provide details unless asked. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
{history}
</conversation_history>

Here is the human's next reply:
<human_reply>
{input}
</human_reply>

Assistant:
""")

conversation2.prompt = claude_prompt2

In [27]:
print_ww(conversation2.predict(input="Hi there!"))

Hello! I'm afraid I don't have any context about our prior conversation to draw from. As an AI
assistant without persistent memory, I can only respond to the most recent statement made. Please
feel free to ask me any questions you may have, and I'll do my best to provide helpful responses
within 3 sentences based on what is stated. If there's any background information that would help me
understand the context, feel free to provide that as well. I look forward to our discussion!


In [28]:
print_ww(conversation2.predict(input="Give me a few tips on how to start a new garden."))

Here are a few tips for starting a new garden in 3 sentences:

Choose a sunny spot with good drainage and easy access to water. Prepare the soil by mixing in
compost or other organic matter to enrich it before planting. Start with easy to grow plants like
tomatoes, peppers, leafy greens, or herbs that need minimal care while you get the hang of
gardening.


In [29]:
print_ww(conversation2.predict(input="Cool. Will that work with tomatoes?"))

Here is a 3 sentence response focusing specifically on growing tomatoes:

Tomatoes thrive in sunny spots with fertile, well-drained soil. Prepare a garden bed by mixing in
compost and working it into the soil to a depth of at least 12 inches before planting. Space tomato
plants 18-24 inches apart and use cages or stakes to support the vines as the fruits develop during
summer.


In [30]:
print_ww(conversation2.predict(input="That's all, thank you!"))

Unfortunately I do not have enough context to provide a meaningful response. As an AI assistant
without persistent memory, I can only respond to the most recent statement made within the bounds of
our conversation. If you have any other questions or information to share, I'm happy to continue our
discussion.


### Interactive session using ipywidgets

The following utility class allows us to interact with Claude in a more natural way. We write out the question in an input box, and get Claude's answer. We can then continue our conversation.

In [31]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat!!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

Let's start a chat. You can also test the following questions:
1. tell me a joke
2. tell me another joke
3. what was the first joke about
4. can you make another joke on the same topic of the first joke

In [32]:
chat = ChatUX(conversation)
chat.start_chat()

Starting chat bot


Output()

---

## Chatbot with persona

AI assistant will play the role of a career coach. Role Play Dialogue requires user message to be set in before starting the chat. ConversationBufferMemory is used to pre-populate the dialog

In [35]:
# store previous interactions using ConversationalBufferMemory and add custom prompts to the chat.
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("You will be acting as a career coach. Your goal is to give career advice to users")
memory.chat_memory.add_ai_message("I am a career coach and give career advice")
cl_llm = BedrockChat(
    model_id="anthropic.claude-v2:1",
    client=boto3_bedrock) # - anthropic.claude-3-sonnet-20240229-v1:0,     anthropic.claude-v2
conversation = ConversationChain(
     llm=cl_llm, verbose=True, memory=memory
)

conversation.prompt = claude_prompt

##print_ww(conversation.predict(input="What are the career options in AI?"))

In [36]:
print_ww(conversation.predict(input="What these people really do? Is it fun?"))



> Entering new ConversationChain chain...
Prompt after formatting:


Human: The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
Human: You will be acting as a career coach. Your goal is to give career advice to users
AI: I am a career coach and give career advice
</conversation_history>

Here is the human's next reply:
<human_reply>
What these people really do? Is it fun?
</human_reply>

Assistant:


> Finished chain.
I do not have enough context to determine specifically what career or "these people" you are
referring to. As a career coach, I aim to provide helpful career guidance without making
assumptions. Could you please clarify what field or career you are asking about? I'm happy to share
my thoughts if you help give me some more details. Please let me know

##### Let's ask a question that is not specialty of this Persona and the model shouldn't answer that question and give a reason for that

In [37]:
conversation.verbose = False
print_ww(conversation.predict(input="How to fix my car?"))

Unfortunately I do not have expertise in auto repair to provide specific advice on fixing cars. As a
career coach, my knowledge is focused on giving guidance related to jobs, education, and planning
one's professional path. If you have any questions about pursuing a career as an auto mechanic or
considerations around jobs in the automotive industry, I'm happy to try and provide some insights.
But for mechanical issues with a specific car, it would be best to consult a professional auto
mechanic. Let me know if you have any other career-related questions I can try to assist with!


## Chatbot with Context 
In this use case we will ask the Chatbot to answer question from some external corpus it has likely never seen before. To do this we apply a pattern called RAG (Retrieval Augmented Generation): the idea is to index the corpus in chunks, then look up which sections of the corpus might be relevant to provide an answer by using semantic similarity between the chunks and the question. Finally the most relevant chunks are aggregated and passed as context to the ConversationChain, similar to providing a history.

We will take a csv file and use **Titan Embeddings Model** to create vectors for each line of the csv. This vector is then stored in FAISS, an open source library providing an in-memory vector datastore. When the chatbot is asked a question, we query FAISS with the question and retrieve the text which is semantically closest. This will be our answer. 

#### Titan embeddings Model

Embeddings are a way to represent words, phrases or any other discrete items as vectors in a continuous vector space. This allows machine learning models to perform mathematical operations on these representations and capture semantic relationships between them.

Embeddings are for example used for the RAG [document search capability](https://labelbox.com/blog/how-vector-similarity-search-works/) 


In [38]:
from langchain.embeddings import BedrockEmbeddings

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

#### FAISS as VectorStore

In order to be able to use embeddings for search, we need a store that can efficiently perform vector similarity searches. In this notebook we use FAISS, which is an in memory store. For permanently store vectors, one can use pgVector, Pinecone or Chroma.

The langchain VectorStore API's are available [here](https://python.langchain.com/en/harrison-docs-refactor-3-24/reference/modules/vectorstore.html)

To know more about the FAISS vector store please refer to this [document](https://arxiv.org/pdf/1702.08734.pdf).

In [39]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.vectorstores import FAISS

s3_path = "s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv"
!aws s3 cp $s3_path ./rag_data/Amazon_SageMaker_FAQs.csv

loader = CSVLoader("./rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars, each row consists in a question column and an answer column
documents_aws = loader.load() #
print(f"Number of documents={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Number of documents after split and chunking={len(docs)}")
vectorstore_faiss_aws = None

    
vectorstore_faiss_aws = FAISS.from_documents(
    documents=docs,
     embedding = br_embeddings
)

print(f"vectorstore_faiss_aws: number of elements in the index={vectorstore_faiss_aws.index.ntotal}::")



download: s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv to rag_data/Amazon_SageMaker_FAQs.csv
Number of documents=153
Number of documents after split and chunking=154
vectorstore_faiss_aws: number of elements in the index=154::


#### Semantic search

We can use a Wrapper class provided by LangChain to query the vector data base store and return to us the relevant documents. Behind the scenes this is only going to run a RetrievalQA chain.

In [40]:
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print_ww(wrapper_store_faiss.query("R in SageMaker", llm=cl_llm))

Yes, R is fully supported in Amazon SageMaker in several ways:

- You can use RStudio on SageMaker to get a managed RStudio IDE in the cloud that can dynamically
scale compute resources. All your R code, data, packages etc. are automatically synced between
RStudio and SageMaker Studio notebooks.

- SageMaker notebook instances come pre-installed with an R kernel and libraries like reticulate to
access the SageMaker Python SDK from R. This allows you to build, train, tune, deploy and manage R
machine learning models using SageMaker.

- You can bring your own Docker container with custom R environments and packages to train and
deploy models on SageMaker.

- SageMaker Experiments provides tracking, visualization and comparison of R model training
experiments.

- SageMaker Automatic Model Tuning can automate hyperparameter tuning for R models to find the best
performing model.

In summary, SageMaker provides a fully managed platform to build, train and deploy R machine
learning models at 

Let's see how the semantic search works:
1. First we calculate the embeddings vector for the query, and
2. then we use this vector to do a similarity search on the store

In [41]:
v = br_embeddings.embed_query("R in SageMaker")
print(v[0:10])
results = vectorstore_faiss_aws.similarity_search_by_vector(v, k=4)
for r in results:
    print_ww(r.page_content)
    print('----')

[1.171875, 0.33398438, 0.3125, -0.24316406, 0.60546875, 0.41992188, -0.36132812, -6.580353e-05, 0.3203125, -0.66796875]
﻿What is Amazon SageMaker?: Is R supported with Amazon SageMaker?
Amazon SageMaker is a fully managed service to prepare data and build, train, and deploy machine
learning (ML) models for any use case with fully managed infrastructure, tools, and workflows.: Yes,
R is supported with Amazon SageMaker. You can use R within SageMaker notebook instances, which
include a preinstalled R kernel and the reticulate library. Reticulate offers an R interface for the
Amazon SageMaker Python SDK, enabling ML practitioners to build, train, tune, and deploy R models.
----
﻿What is Amazon SageMaker?: What is RStudio on Amazon SageMaker?
Amazon SageMaker is a fully managed service to prepare data and build, train, and deploy machine
learning (ML) models for any use case with fully managed infrastructure, tools, and workflows.:
RStudio on Amazon SageMaker is the first fully managed RSt

#### Memory
In any chatbot we will need a QA Chain with various options which are customized by the use case. But in a chatbot we will always need to keep the history of the conversation so the model can take it into consideration to provide the answer. In this example we use the [ConversationalRetrievalChain](https://python.langchain.com/docs/modules/chains/popular/chat_vector_db) from LangChain, together with a ConversationBufferMemory to keep the history of the conversation.

Source: https://python.langchain.com/docs/modules/chains/popular/chat_vector_db

Set `verbose` to `True` to see all the what is going on behind the scenes.

In [42]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

print_ww(CONDENSE_QUESTION_PROMPT.template)

Given the following conversation and a follow up question, rephrase the follow up question to be a
standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:


#### Parameters used for ConversationRetrievalChain
* **retriever**: We used `VectorStoreRetriever`, which is backed by a `VectorStore`. To retrieve text, there are two search types you can choose: `"similarity"` or `"mmr"`. `search_type="similarity"` uses similarity search in the retriever object where it selects text chunk vectors that are most similar to the question vector.

* **memory**: Memory Chain to store the history 

* **condense_question_prompt**: Given a question from the user, we use the previous conversation and that question to make up a standalone question

* **chain_type**: If the chat history is long and doesn't fit the context you use this parameter and the options are `stuff`, `refine`, `map_reduce`, `map-rerank`

If the question asked is outside the scope of context, then the model will reply it doesn't know the answer

**Note**: if you are curious how the chain works, uncomment the `verbose=True` line.

In [43]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    memory=memory_chain,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    #verbose=True, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

Let's chat! ask the chatbot some questions about SageMaker, like:
1. What is SageMaker?
2. What is canvas?

In [44]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

Your mileage might vary, but after 2 or 3 questions you will start to get some weird answers. In some cases, even in other languages.
This is happening for the same reasons outlined at the beginning of this notebook: the default langchain prompts are not optimal for Claude. 
In the following cell we are going to set two new prompts: one for the question rephrasing, and one to get the answer from that rephrased question.

In [49]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import BaseMessage


# We are also providing a different chat history retriever which outputs the history as a Claude chat (ie including the \n\n)
_ROLE_MAP = {"human": "\n\nHuman: ", "ai": "\n\nAssistant: "}
def _get_chat_history(chat_history):
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "\n\nHuman: " + dialogue_turn[0]
            ai = "\n\nAssistant: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer

# the condense prompt for Claude
condense_prompt_claude = PromptTemplate.from_template("""{chat_history}

Answer only with the new question.


Human: How would you ask the question considering the previous conversation: {question}


Assistant: Question:""")

# recreate the Claude LLM with more tokens to sample - this provides longer responses but introduces some latency
cl_llm = BedrockChat(model_id="anthropic.claude-v2:1", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 500})
memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    get_chat_history=_get_chat_history,
    #verbose=True,
    condense_question_prompt=condense_prompt_claude, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

# the LLMChain prompt to get the answer. the ConversationalRetrievalChange does not expose this parameter in the constructor
qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}

Human: Use at maximum 3 sentences to answer the question inside the <q></q> XML tags. 

<q>{question}</q>

Do not use any XML tags in the answer. If the answer is not in the context say "Sorry, I don't know as the answer was not found in the context"

Assistant:""")

Let's start another chat. Feel free to ask the following questions:

1. What is SageMaker?
2. what is canvas?

In [50]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

#### Do some prompt engineering

You can "tune" your prompt to get more or less verbose answers. For example, try to change the number of sentences, or remove that instruction all-together. You might also need to change the number of `max_tokens` (eg 1000 or 2000) to get the full answer.

### In this demo we used Claude V3 sonnet LLM to create conversational interface with following patterns:

1. Chatbot (Basic - without context)

2. Chatbot using prompt template(Langchain)

3. Chatbot with personas

4. Chatbot with context